In [1]:
pip install ultralytics opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.0 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
import cv2

# Load pretrained YOLO model
model = YOLO("yolov5s.pt")

# Read image
image = cv2.imread("construction_site.jpg")

# Run detection
results = model(image)

# Display detected objects
for r in results:
    for box in r.boxes:
        cls = int(box.cls[0])
        label = model.names[cls]
        print("Detected Material:", label)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.12/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.12/dist-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 869.2ms
image 2/2 /usr/local/lib/python3.12/dist-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 2 ties, 551.4ms
Speed: 11.6ms preprocess, 710.3ms inference, 25.2ms postprocess per image at shape (1, 3, 384, 6

In [4]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
import numpy as np

# Load pretrained model
model = resnet50(pretrained=True)
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

# Create a dummy image if it doesn't exist
# This is to ensure the code runs even if 'concrete_sample.jpg' is not provided
try:
    img = Image.open("concrete_sample.jpg")
except FileNotFoundError:
    print("Creating a dummy 'concrete_sample.jpg' as it was not found.")
    dummy_image_data = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
    img = Image.fromarray(dummy_image_data)
    img.save("concrete_sample.jpg")

# Load image (or use the newly created dummy image)
img = Image.open("concrete_sample.jpg")
img = transform(img).unsqueeze(0)

# Prediction
with torch.no_grad():
    output = model(img)

predicted_class = torch.argmax(output, 1)
print("Material Class ID:", predicted_class.item())


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Creating a dummy 'concrete_sample.jpg' as it was not found.
Material Class ID: 610


In [5]:
import torch.nn as nn

feature_extractor = resnet50(pretrained=True)
feature_extractor.fc = nn.Identity()
feature_extractor.eval()

features = feature_extractor(img)
print("Extracted Feature Vector Shape:", features.shape)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Extracted Feature Vector Shape: torch.Size([1, 2048])


In [6]:
import torch
import torch.nn as nn

class ImageCaptioningModel(nn.Module):
    def __init__(self, feature_size, hidden_size, vocab_size):
        super().__init__()
        self.lstm = nn.LSTM(feature_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, features):
        features = features.unsqueeze(1)
        outputs, _ = self.lstm(features)
        captions = self.fc(outputs)
        return captions


In [7]:
# Example vocabulary
vocab = {
    0: "Concrete structure",
    1: "Steel reinforcement",
    2: "Brick masonry",
    3: "Multi-storey building under construction"
}

predicted_index = 3
print("Generated Description:", vocab[predicted_index])


Generated Description: Multi-storey building under construction


In [8]:
def generate_report(materials, stage):
    report = f"""
    Project Progress Report:
    ------------------------
    Current Stage: {stage}

    Identified Materials:
    """
    for m in materials:
        report += f"- {m}\n"

    report += """
    Construction Method:
    Reinforced concrete framed structure.

    Next Phase:
    Slab casting and vertical expansion.
    """
    return report


In [9]:
materials = ["Concrete", "Steel", "Bricks"]
stage = "Foundation and Column Completion"

print(generate_report(materials, stage))



    Project Progress Report:
    ------------------------
    Current Stage: Foundation and Column Completion

    Identified Materials:
    - Concrete
- Steel
- Bricks

    Construction Method:
    Reinforced concrete framed structure.

    Next Phase:
    Slab casting and vertical expansion.
    
